# EkstraModel

In [8]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Wyświetl/schowaj kod."></form>''')

### Aby uruchomić, wciśnij spację

In [9]:
import ipywidgets as widgets
from ipywidgets import Layout

style = {'description_width': 'initial'}

In [56]:
n = widgets.FloatLogSlider(value=10000, base=10, min=1, max=10, step=1, description='Liczba symulacji', style=style)
matchday = widgets.BoundedIntText(value=1, min=1, max=34, step=1, description='Wybierz kolejkę:', disabled=False, style=style)
league = widgets.Dropdown(options=['ekstraklasa', '1liga'], value='ekstraklasa', description='Liga:', disabled=False)

display(n)
display(league)
display(matchday)

button = widgets.Button(description='Zatwierdź',disabled=False,tooltip='Confirm',icon='check') 
output = widgets.Output()
display(button,output)

FloatLogSlider(value=10000.0, description='Liczba symulacji', max=10.0, min=1.0, step=1.0, style=SliderStyle(d…

Dropdown(description='Liga:', options=('ekstraklasa', '1liga'), value='ekstraklasa')

BoundedIntText(value=1, description='Wybierz kolejkę:', max=34, min=1, style=DescriptionStyle(description_widt…

Button(description='Zatwierdź', icon='check', style=ButtonStyle(), tooltip='Confirm')

Output()

In [63]:
import os
def initialize(now):
    #structures for collecting data: goals shot and conceded, no. of matches played home and away, whole league shot and conceded
    team_shot_home = {}
    team_conceded_home = {}
    team_shot_away = {}
    team_conceded_away = {}
    home_teams = []
    away_teams = []
    league_stats = {'shot_home_avg': 0, 'shot_away_avg':0}

    #appending data to strcutures for all matchdays till the chosen one
    matchdays_played = [int(m) for m in os.listdir(f"{league.value}_matchdays") if int(m)<int(now)]
    matches_no = 0
    for m in matchdays_played:
        h = open(f"{league.value}_matchdays/{m}", "r", encoding='utf-8')
        for match in h:
            matches_no+=1
            home, away, goal_home, goal_away = match.strip().split("-")
            goal_home, goal_away = int(goal_home), int(goal_away)
            home_teams.append(home)
            away_teams.append(away)

            league_stats['shot_home_avg']+=goal_home
            league_stats['shot_away_avg']+=goal_away
            
            if not home in team_shot_home:
                team_shot_home[home]=goal_home
                team_conceded_home[home]=goal_away
            else:
                team_shot_home[home]+=goal_home
                team_conceded_home[home]+=goal_away
            if not away in team_shot_away:
                team_shot_away[away]=goal_away
                team_conceded_away[away]=goal_home
            else:
                team_shot_away[away]+=goal_away
                team_conceded_away[away]+=goal_home
            
        h.close()
        
    league_stats["shot_home_avg"]/=matches_no
    league_stats["shot_away_avg"]/=matches_no

    home_att_strength = {}
    home_def_strength = {}
    away_att_strength = {}
    away_def_strength = {}

    #calucalting defence and attack strength for each team
    for k in team_shot_home.keys():
        k_home = home_teams.count(k)
        k_away = away_teams.count(k)
        home_att_strength[k] = (team_shot_home[k]/k_home)/league_stats['shot_home_avg']
        home_def_strength[k] = (team_conceded_home[k]/k_home)/league_stats['shot_away_avg']
        away_att_strength[k] = (team_shot_away[k]/k_away)/league_stats['shot_away_avg']
        away_def_strength[k] = (team_conceded_home[k]/k_away)/league_stats['shot_home_avg']

    return league_stats, home_att_strength, home_def_strength, away_att_strength, away_def_strength

In [64]:
import numpy as np
from collections import Counter
from IPython.display import clear_output

def display_matchday(a):
    clear_output()
    league_stats, home_att_strength, home_def_strength, away_att_strength, away_def_strength = initialize(a)
    current = open(f"{league.value}_matchdays/{str(a)}", "r", encoding='utf-8')
    for match in current:
        home_team = match.strip().split("-")[0]
        away_team = match.strip().split("-")[1]
        rng = np.random.default_rng()
        results = []
        home = home_att_strength[home_team]*away_def_strength[away_team]*league_stats['shot_home_avg']
        away = home_def_strength[home_team]*away_att_strength[away_team]*league_stats['shot_away_avg']
        print(home_team, "vs", away_team)
        print("Siła:", home, away)
        for i in range(0, int(n.value)):
            results.append((int(rng.poisson(home, 1)[0]),int(rng.poisson(away, 1)[0])))
        print("Prawdopodobieństwo czystego konta gospodarza:", round(sum(x[1]==0 for x in results)/n.value, 4))
        print("Prawdopodobieństwo czystego konta gościa:", round(sum(x[0]==0 for x in results)/n.value, 4))
        
        print("Najbardziej prawdopodobne wyniki:\n", Counter(results).most_common(7),"\n\n\n")
    current.close()

In [65]:
def on_confirm_button_clicked(b):
    with output:
        display_matchday(matchday.value)
    
button.on_click(on_confirm_button_clicked)